In [ ]:
import pandas as pd

In [ ]:
import json
import pandas as pd
import cv2
import numpy as np
from enum import Enum, auto
import sys
import torch
import matplotlib.pyplot as plt

sys.path.append("../Skin_segmentation")
sys.path.append("../../Models")

from Skindetector import *
from rPPGNet import rPPGNet

with open("../../Data/json_structure") as json_file:
    data = json.load(json_file)


root_dir = "/work3/s174159/data/"
video_file = root_dir + "15/01/c920-1.avi"
bb_file = root_dir + "bbox/15/01/c920-1.face"
ecg_path = root_dir + "15/01/viatom-raw.csv"
index_path = root_dir + "15/01/c920.csv"

bb_data = pd.read_csv(
    bb_file, sep=" ", header=None, names=["frame", "x", "y", "w", "h"]
).drop("frame", axis=1)

In [ ]:
df = pd.read_csv(ecg_path)
df.head(5)

In [ ]:
lead_names = ["Lead I", "Lead II", "Lead III", "Lead aVR", "Lead aVL"]

In [ ]:
df["Lead"] = lead_names * (len(df) // 5)
df["time"] = (df["milliseconds"] - df["milliseconds"].min()) / 1000  # Convert time.
tmp_df = df[df["Lead"] == "Lead II"]
tmp_df = tmp_df[tmp_df["time"] < 60]
tmp_df[" ECG"] = tmp_df[" ECG"] * 1e-8

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(tmp_df["time"], tmp_df[" ECG"])
plt.xlim(0, 6)

In [ ]:
def moving_average(series, window_size=5):
    # Convert array of integers to pandas series

    # Get the window of series
    # of observations of specified window size
    windows = series.rolling(window_size)

    # Create a series of moving
    # averages of each window
    moving_averages = windows.mean()

    # Convert pandas series back to list
    moving_averages_list = moving_averages.tolist()
    final_list = moving_averages_list
    return final_list


tmp_df["ECG NORM"] = (tmp_df[" ECG"] - np.min(tmp_df[" ECG"])) / (
    np.max(tmp_df[" ECG"]) - np.min(tmp_df[" ECG"])
)
tmp_df["ECG MV"] = moving_average(tmp_df["ECG NORM"], 3)

In [ ]:
def moving_average(series, window_size=5):
    # Convert array of integers to pandas series

    # Get the window of series
    # of observations of specified window size
    windows = series.rolling(window_size)

    # Create a series of moving
    # averages of each window
    moving_averages = windows.mean()

    # Convert pandas series back to list
    moving_averages_list = moving_averages.tolist()
    final_list = moving_averages_list
    return final_list


tmp_df["ECG NORM"] = (tmp_df[" ECG"] - np.min(tmp_df[" ECG"])) / (
    np.max(tmp_df[" ECG"]) - np.min(tmp_df[" ECG"])
)
tmp_df["ECG MV"] = moving_average(tmp_df["ECG NORM"], 3)

In [ ]:
plt.figure(figsize=(20, 10))
# plt.plot(tmp_df["time"], tmp_df[" ECG"])
plt.plot(tmp_df["time"], tmp_df["ECG NORM"])
plt.plot(tmp_df["time"], tmp_df["ECG MV"])
plt.xlim(0, 6)